# LTKHDL - Final Project

(Cập nhật lần cuối: 10/1/2022)
<h3> Thành viên nhóm: </h3>
 - Nguyễn Tuấn Anh - 18120110 <br>
 - Chương Việt Hoàng - 18120124 <br>
 - Lê Thị Tuyết Trinh - 18120616 <br>


---

## Môi trường code

In [ ]:
import sys
sys.executable

Nếu không có vấn đề gì thì file chạy python sẽ là file của môi trường code "min_ds-env".

---

## Import các thư viện cần thiết

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# YOUR CODE HERE (OPTION) 

---

## Thu thập dữ liệu

Trong đồ án này, nhóm em sửa dụng dữ liệu đã được thu thập sẵn là file "review_mbbank.csv" đính kèm; đây là dữ liệu về đánh giá của người dùng về app MB Bank. Dữ liệu này được crawl từ Google Play và gốc của dữ liệu là [ở đây]().

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [ ]:
df_review = pd.read_csv('review_mbbank.csv')
df_review

### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

Tính số dòng và số cột (không được code cứng con số) và lần lượt lưu vào 2 biến là `num_rows` và `num_cols`. 

In [ ]:
# YOUR CODE HERE
num_rows, num_cols = df_review.shape

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Đây là dữ liệu được crawl từ bên Google Play về với nội dung là review của khách hàng nên nhóm em nghĩ sẽ có nhiều vấn đề ví dụ như bị trùng lặp dữ liệu, kiểu dữ liệu,...

### Dữ liệu có các dòng bị lặp không?

Bạn sẽ kiểm tra vụ này và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu ngược. Gợi ý: dataframe có phương thức `duplicated`.

In [ ]:
# YOUR CODE HERE
#TODO: Check have_duplicated_rows
have_duplicated_rows = df_review.duplicated().any()
print(have_duplicated_rows)

### Mỗi cột có ý nghĩa gì?

Dưới đây là phần mô tả của dữ liệu:
- **ID**: a unique identifier for each observation.
- **reviewer**: the account's name.
- **content**: the content of reviewer.
- **date_time**: date the reviewer take review.
- **rate**: rating of reviewer for app.

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Nhóm em lấy `dtype` (kiểu dữ liệu của mỗi phần tử) của mỗi cột trong `df_review` và lưu kết quả vào series `col_dtypes`; series này có index là tên cột. 

In [ ]:
# YOUR CODE HERE
col_dtypes = df_review.dtypes
print(col_dtypes)

**Cột có dtype là object nghĩa là sao?**

`df_review` có 3 cột có `dtype` là `object`: reviewer, content, date_time
Chúng em viết hàm nhận đầu vào là một cột `s` (series) có `dtype` là `object` và trả về tập các kiểu dữ liệu thật sự có trong cột này. (Sử dụng series.apply())

In [ ]:
def open_object_dtype(s):
    dtypes = set()
    # YOUR CODE HERE
    col = pd.Series(s)
    result = col.apply(lambda x: type(x))
    dtypes = set(result)
    return dtypes
print(open_object_dtype(df_review['reviewer']))
print(open_object_dtype(df_review['content']))
print(open_object_dtype(df_review['date_time']))

**Vấn đề cần tiền xử lý:**

- Cột "date_time" đang có dtype là `str`. Để có thể tiếp tục khám phá thêm về cột này, ta sẽ thực hiện bước tiền xử lý là chuyển sang dạng `datetime`.
- Cột "content" đang có dtype là `str`, cần kiểm tra xem có dòng content nào là chuỗi rỗng không (vì đánh giá trên google play người dùng có thể nhập chuỗi gồm các khoảng trắng và submit được).
- Cột "reviewer" đang có dtype là `str`, có lẽ cột này không có giá trị rỗng nào

---

## Tiền xử lý

### Chuyển dtype của cột "date_time" sang datetime (0.5đ)

Nhóm em chuyển `dtype` của cột "date_time" sang kiểu dữ liệu thời gian (sau khi làm xong thì cột "date_time" trong `df_review` phải thay đổi `dtype`). Nhóm em dùng câu lệnh `pd.to_datetime`; khi dùng `pd.to_datetime`, bạn truyền giá trị phù hợp cho tham số `format` (nếu không thì sẽ chạy rất chậm vì Pandas phải đoán format).

In [ ]:
# YOUR CODE HERE
df_review['date_time'] = pd.to_datetime(df_review['date_time'], format="%d-%m-%y %H:%M")
# Nếu dtype của cột "Date" đã được chuyển sang datetime thì phải dùng được .dt.
assert df_review['date_time'].dt.year.min() == 2021 

### Chuyển các giá trị chuỗi trống (chỉ chứa khoảng trắng) sang NaN

Nhóm em sẽ chuyển các giá trị trống (nếu có) của cột "content" sang NaN, vì thực tế người dùng khi đánh giá app có thể nhập chuỗi trống vào và submit được.

In [ ]:
# YOUR CODE HERE (x.isspace() or not x)
#replace all string " " in column District and CommunityArea
df_review['content'] = df_review['content'].replace(r'^\s+$', np.nan, regex=True)
print(df_review['content'].isna().sum())


Có thể thấy, không có dòng nào trong content là rỗng, bây giờ chúng em sẽ thực hiên khám phá dữ liệu và trả lời câu hỏi có ý nghĩa.

### Phân tích năm 

In [ ]:
# YOUR CODE HERE
col_year = df_review['date_time'].dt.year
col_year = set(col_year)
print(col_year)

Em lấy các năm trong thời gian review thì thấy các review này đều được lấy trong năm 2021 vậy nên tạm thời về năm không có nhiều thứ cần khai thác, nên nhóm em sẽ lấy ra các tháng để phân tích sâu hơn.

### Thêm cột "Month" vào dataframe (0.5đ)

In [ ]:
# YOUR CODE HERE
df_review['Month'] = df_review['date_time'].dt.month
df_review.head()

### Tự thêm các điều cần xử lý ở đây

Lý giải các điều cần xử lý

---

## Khám phá dữ liệu 

### Trong cột rate, số lượng mỗi điểm là bao nhiêu?

Cột rate chứa điểm số mà người dùng đánh giá cho app, viết hàm để tính số lượng từng điểm (1 - 5) và vẽ biểu đồ

In [ ]:
# YOUR CODE HERE
#Hàm để tính min, max, missing_ratio của dfcol được truyền vào
rate_count = df_review['rate'].value_counts()
rate_count = rate_count.sort_index()
print(rate_count)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(rate_count.index,rate_count)
plt.show()

Nhìn biểu đồ có thể thấy số lượt rate 1 sao và 5 sao cho app là chiếm đa số, trong đó app bị rate 1 sao nhiều nhất (6699 người đánh giá 1 sao).

### Tự thêm các điều cần khai thác ở đây

Lý giải các điều cần khai thác

In [ ]:
# YOUR CODE HERE


---

## Đưa ra các câu hỏi cần trả lời

Sau khi đã khám phá dữ liệu và hiểu hơn về dữ liệu, ta thấy có một số câu hỏi có thể được trả lời bằng dữ liệu:
- .... (Đưa ra câu hỏi) -- Giải thích tại sao lại là câu hỏi có ý nghĩa


---

## Tiền xử lý

### Tự thêm các điều cần tiền xử lý

Lý giải điều cần tiền xử lý

In [ ]:
# YOUR CODE HERE


---

## Phân tích dữ liệu để trả lời cho các câu hỏi 

### Title -- câu hỏi của Nguyễn Tuấn Anh

Để trả lời cho câu hỏi này, bạn sẽ làm như sau:
- Bước 1: ...
- Bước 2: ...

Code bước 1. 

In [ ]:
# YOUR CODE HERE


Code bước 2. 

In [ ]:
# YOUR CODE HERE


Nhận xét: ...

### Title -- câu hỏi của Chương Việt Hoàng

Để trả lời cho câu hỏi này, bạn sẽ làm như sau:
- Bước 1: ...
- Bước 2: ...

Code bước 1. 

In [ ]:
# YOUR CODE HERE


Nhận xét: 

Code bước 2.

In [ ]:
# YOUR CODE HERE

### Title -- câu hỏi của Lê Thị Tuyết Trinh

Để trả lời cho câu hỏi này, bạn sẽ làm như sau:
- Bước 1: ...
- Bước 2: ...

Code bước 1. 

In [ ]:
# YOUR CODE HERE

Code bước 2. 

In [ ]:
# YOUR CODE HERE

Nhận xét: ...